<a href="https://colab.research.google.com/github/sungjin-kim-data/vgames2_data/blob/master/vgames2_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **성진이가 다니고 있는 게임 회사**

## **게임 데이터를 확인 후 액션이 어떻게 이루어져야하는지 확인할 것**

1. 지역에 따라서 선호하는 게임 장르가 다를까 라는 질문에 대답
2. 연도별 게임의 트렌드가 있을까 라는 질문에 대답
3. 출고량의 높은 게임에 대한 분석 및 시각화 프로세스 포함

=> 결론 : 다음 분기에 어떤 게임을 설계해야 하는지를 제시하셔야 합니다.

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

plt.style.use('seaborn')
sns.set(font_scale=1)
plt.rc('font', family='NanumBarunGothic')
plt.rcParams['font.family'] = 'NanumGothic'

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
# import matplotlib.font_manager as fm
# fm.fontManager.ttflist
# [f.name for f in fm.fontManager.ttflist]

In [ ]:
# 데이터 불러오기
gd = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/datasets/vgames2.csv')
gd.head()

In [ ]:
# columns 소문자로 변경

gd.columns = gd.columns.str.lower()
gd.columns

In [ ]:
gd

columns

  - Name : 게임의 이름
  - Platform : 게임이 지원되는 플랫폼의 이름
  - Year : 게임이 출시된 연도
  - Genre : 게임의 장르
  - Publisher : 게임을 제작한 회사
  - NA_Sales : 북미지역에서의 출고량
  - EU_Sales : 유럽지역에서의 출고량
  - JP_Sales : 일본지역에서의 출고량
  - Other_Sales : 기타지역에서의 출고량

## **데이터 확인 및 정제**

In [ ]:
# 필요없는 컬럼 제거
gd.drop('unnamed: 0', axis=1,inplace=True)

In [ ]:
# 정보 확인
gd.info()

In [ ]:
# object describe
gd.describe(include=np.object)

In [ ]:
# gd 내용 정리해서 출력

print("총 데이터 수 :", gd.shape[0]*gd.shape[1])
print("총 결측치 수 : {} / 전체 데이터의 {:.2f}%".format(gd.isnull().sum().sum(), (gd.isnull().sum().sum()*100)/(gd.shape[0]*gd.shape[1])))
print("전체 게임 수: ", gd.name.nunique())
print("전체 플랫폼의 수: ", gd.platform.nunique())

In [ ]:
# 결측치 확인
gd.isnull().sum(axis=0)

In [ ]:
# 총 379개의 결측치 존재 전체 데이터의 0.25% 차지 그렇기에 결측치를 제거하여도 분석에 그렇게 크게 영향을 미칠것으로 판단되지 않음
# 즉, 결측치는 제거하는 것으로 판다

In [ ]:
# 결측치 컬럼 내용 확인
gd[gd.isnull().any(axis=1)].head()

In [ ]:
gd1 = gd.dropna()
gd1 = gd1.reset_index(drop=True)
gd1.info()

In [ ]:
# 결측치 정상 제거 확인
gd1.isnull().sum()

In [ ]:
# 장르별 게임 수
gd1['genre'].value_counts()

In [ ]:
# 장르별 게임 수 시각화

plt.figure(figsize=(15, 10))
sns.countplot(x="genre", data=gd1, order = gd1['genre'].value_counts().index)
# plt.title('장르별 게임 출시 수량')
plt.show()

In [ ]:
# 이 시각화로 알 수 있는건 액션장르의 게임이 압도적으로 많다는 사실이다.

In [ ]:
# Year int로 변경
gd1['year'] = gd1['year'].astype('int64')
gd1.head()

In [ ]:
gd1.info()

In [ ]:
# 음수 값 확인
print(min(gd1['na_sales']), max(gd1['na_sales']))
print(min(gd1['eu_sales']), max(gd1['eu_sales']))
print(min(gd1['jp_sales']), max(gd1['jp_sales']))
print(min(gd1['other_sales']), max(gd1['other_sales']))

In [ ]:
# Year int로 변경
gd1['year'] = gd1['year'].astype('int64')
gd1

In [ ]:
# year 정제
gd1.query('year < 10')

In [ ]:
# year 잘못 기입된 index 추출
year = gd1.query('year < 10').index
year

In [ ]:
# 2000 추가
gd1.loc[year, 'year'] = 2000 + gd1.loc[year, 'year']

In [ ]:
# 2022 기준으로 2자리 숫 year 출력
gd1.query('year < 22')

In [ ]:
year2 = gd1.query('year < 22').index
year2

In [ ]:
# 2000 추가
gd1.loc[year2, 'year'] = 2000 + gd1.loc[year2, 'year']

In [ ]:
gd1.query('year < 22')

In [ ]:
# 마지막으로 1900년대 2자리 숫자 출력
gd1.query('year < 100')

In [ ]:
year3 = gd1.query('year < 100').index
year3

In [ ]:
gd1.loc[year3, 'year'] = 1900 + gd1.loc[year3, 'year']

In [ ]:
gd1.loc[year3]

### 이제 sales를 정제하려고 한다 엑셀 대충 데이터를 확인 했을 때 K, M 
### 그리고 0.xx 같은 통합되어 있지 않은 데이터들이라 전부 통일시켜준다

In [ ]:
# salse의 경우 전부 백만 단위로 변경

# 먼저 na_sales 부터 시작
# 뒤에 K가 붙는 row 추출
gd1[gd1['na_sales'].str.endswith('K') == True]

# 0K라고 추출 했을 때 0K는 사실 뒤에 K라는 단위가 붙지만 첫번째 자리가 0인 것을 감안해 무의미한 데이터라고 간부 'K'를 전부

In [ ]:
plt.figure(figsize=(15, 10))
sns.countplot(x="year", data=gd1, order = gd1.groupby(by=['year'])['name'].count().sort_values(ascending=False).index)
plt.title('연도별 게임 출시 현황', fontsize=15)
plt.xlabel('연도', rotation=0)
plt.ylabel('수량', rotation=0)
plt.xticks(rotation=45)
plt.show()

In [ ]:
plt.figure(figsize=(30, 10))
sns.countplot(x="year", data=gd1, hue='genre', order=gd1.year.value_counts().iloc[:10].index)
plt.title('게임 장르별 Top10 연도별 정리', fontsize=20)
plt.xlabel('연도')
plt.ylabel('수량', rotation=0)
plt.xticks(rotation=90)
plt.show()

In [ ]:
# 2002년을 제외한 Top10 연도는 전부 액션 게임이 1등이다.